# TPK4116 - Value Chain Analysis

# Semester Assignemt 

| Contributors |
| :- |
| Ida Bjørn-Hansen Ahlsand |
| Tinus Alsos |
| Thomas Willums | 

## Problem 1 (Python)

### a) 

First of all, we decide the six most obvious decision variables, representing hours of production for product 1 and 2 for machines A, B and C. Let H/d be hours per day and P1 and P2 denote product 1 and 2 respectively.

| Decision variable | Desciption | Constraint |
| :-: | :- | --- |
| $x_1$ | H/d on machine A to produce P1| $\geq 0$ |
| $x_2$ | H/d on machine B to produce P1| $\geq 0$ |
| $x_3$ | H/d on machine C to produce P1| $\geq 0$ |
| $x_4$ | H/d on machine A to produce P2| $\geq 0$ |
| $x_5$ | H/d on machine B to produce P2| $\geq 0$ |
| $x_6$ | H/d on machine C to produce P2| $\geq 0$ |

Due to assumption that we for each product process a batch of units for each machine one day, we have that the total produced per prodct on each machine for each must be equal. That is, for P1, we have:

$$ \mu_{A1}x_1 = \mu_{B1}x_2 = \mu_{C1}x_3, $$

and for P2 we have:

$$ \mu_{A2}x_4 = \mu_{B2}x_5 = \mu_{C2}x_6. $$

Since our objective is to maximize profit, we simply maximize profit per product times number of items produced per product type. That is, the objective function is:

$$ Maximize: Z = p_1\mu_{A1}x_1 + p_2\mu_{A2}x_4. $$

Obviously, we could substitute the $x's$ and $\mu's$ by the above equalities (constraints).

Now there are two different categories of constraints to handle, namely the sequence of machines, and the possible production time during one day (8 hours).

**Sequence:**
P1 has sequence A - B - C while P2 has sequence B - C - A. 
* When we start production for P1, we begin by utilizing machine A. After P1 is finished with A, it will need to use machine B. Machine B could be occupied by P2, therefore, it is a potential wait time for P1 to use machine B which we denote $x_7$. After P1 is finished with B, it will need to use machine C. Machine C could be occupied by P2, therefore, it is a potential wait time for  P1 to use machine C which we denote $x_8$.
* When we start production for P2, we begin by utilizing machine B. After P2 is finished with B, it will need to use machine C. Machine C could be occupied by P1, therefore, it is a potential wait time for P2 to use machine C which we denote $x_9$. After P2 is finished with C, it will need to use machine A. Machine A could be occupied by P1, therefore, it is a potential wait time for P2 to use machine B which we denote $x_{10}$.

Note that the profits and and production rates are generally balanced, so it is unlikely that $x_i, i = 8, 9, 10$ will be significant. Also note that in theory, we could add slack variables for P1 to wait for machine A and for P2 to wait for machine B.

Adding these (slack) variables as decision variables, we get the following table:


| Decision variable | Desciption | Constraint |
| :-: | :- | --- |
| $x_1$ | H/d on machine A to produce P1| $\geq 0$ |
| $x_2$ | H/d on machine B to produce P1| $\geq 0$ |
| $x_3$ | H/d on machine C to produce P1| $\geq 0$ |
| $x_4$ | H/d on machine A to produce P2| $\geq 0$ |
| $x_5$ | H/d on machine B to produce P2| $\geq 0$ |
| $x_6$ | H/d on machine C to produce P2| $\geq 0$ |
| $x_7$ | H/d P1 needs before P2 is finished using machine B| $\geq 0$ |
| $x_8$ | H/d P1 needs before P2 is finished using machine C| $\geq 0$ |
| $x_9$ | H/d P2 needs before P1 is finished using machine C| $\geq 0$ |
| $x_{10}$ |H/d P2 needs before P1 is finished using machine A| $\geq 0$ |

With the slack variables introduced, we can formalize the limitzations posed by the sequence. We use the first constraint in the first bullet point above as an example.

* When we start production for P1, we begin by utilizing machine A. After P1 is finished with A, it will need to use machine B. Machine B could be occupied by P2, therefore, it is a potential wait time for P1 to use machine B which we denote  𝑥7.

This means that $$x_1 + x_7 \geq x_4,$$ that is, the time before P1 can use machine B to produce (which it will do after it has produced on machine A and potentially waited) must be greater than or equal to the time P2 spend to produce on machine B. We assume that both machines will begin producing instantly.

Doing the same for the next sequence constraints, we get:
$$x_1 + x_2 + x_7 + x_8 \geq x_5 + x_6.$$

Now, the last constraints we need to add are the time limits each day. Clearly, we can spend a maximum of 8 hours producing (including wait time) for each product. For P1, we get:
$$ x_1 + x_2 + x_3 + x_7 + x_8 \leq 8 $$

and for P2, we get:

$$ x_4 + x_5 + x_6 + x_9 + x_{10} \leq 8 $$

Note that we can safely substitue $\leq$ with $=$ since we do not have any costs, the proftits are positive, and we are free to produce fractions (we have not added a constraint that the $x's$ multiplied by respective production rates, $\mu's$ need to result in integers). That is, producing more will always be benefitial.

***

#### b) 

To find the optimal solution, we utilize Gurobi, a state-of-the-art mathematical optimization software for LP-problems.

In [1]:
# Imports
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from gurobipy import LinExpr

In [2]:
# data
mua1 = 16
mua2 = 8
mub1 = 10
mub2 = 8
muc1 = 14
muc2 = 6
p1, p2 = 10, 12

In [3]:
m = gp.Model() # Initiate empty model

# Decision variables
x = m.addVars(range(1, 11), vtype = 'C', name = 'x', lb = 0) 
# initiate x's with index from 1 to 10 with lower bound 0 and type continous

Z = LinExpr([p1, p2], [x[1], x[4]]) # Objective function
m.setObjective(Z, GRB.MAXIMIZE) # Mazimize Z

# Equal batch sizes for P1
m.addConstr(x[1]*mua1 == x[2]*mub1) 
m.addConstr(x[2]*mub1 == x[3]*muc1)

# Equal batch sizes for P2
m.addConstr(x[4]*mua2 == x[5]*mub2)
m.addConstr(x[5]*mub2 == x[6]*muc2)

m.addConstr(sum(x[i] for i in range(1, 4)) + x[7] + x[8] <= 8)
m.addConstr(sum(x[i] for i in range(4,7)) + x[9] + x[10] <= 8)

# Sequence constraints
m.addConstr(x[1] + x[7] >= x[4])
m.addConstr(x[1] + x[2] + x[7] + x[8] >= x[5] + x[6])

m.update()
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 10 columns and 27 nonzeros
Model fingerprint: 0xdf0b067a
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 8e+00]
Presolve removed 5 rows and 6 columns
Presolve time: 0.02s
Presolved: 3 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0174046e+01   7.633588e-02   0.000000e+00      0s
       1    4.9216667e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds (0.00 work units)
Optimal objective  4.921666667e+01


In [4]:
# Results
res = m.getAttr('x', x)
res_dic = {}
for i in range(1, 11):
    res_dic['x_' + str(i)] = x[i].x
df_res = pd.DataFrame.from_dict(res_dic, orient = 'index')
df_res.index.name = 'Decision Variables'
display(df_res)

,0
Decision Variables,
x_1,2.041667
x_2,3.266667
x_3,2.333333
x_4,2.400000
x_5,2.400000
x_6,3.200000
x_7,0.358333
x_8,0.000000
x_9,0.000000


***

#### c) 

Shadow prices is the change per infitesimal unit of the contraint. In our case, the shadow price is how much more profit we would get each hours (or any small time unit) we add to the workday. We have three cases which we can calculated shadow prices for. See excel version for shadow prices.